In [19]:
import sys
sys.path.insert(0, '/home/hengji/Documents/three_kingdoms_social_network/scrape_novel/people_scraper/')

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
from langconv import *
from collections import defaultdict

In [20]:
base_url = 'https://en.wikipedia.org'

def Traditional2Simplified(sentence):
    sentence = Converter('zh-hans').convert(sentence)
    return sentence

In [21]:
# Parse page
html = urlopen('https://en.wikipedia.org/wiki/Lists_of_people_of_the_Three_Kingdoms')
bs = BeautifulSoup(html, 'html.parser')
links = bs.find('td', {'class':"navbox-list navbox-odd hlist"}).find_all(
    href=re.compile('^(/wiki/List_of_people_of_the_Three_Kingdoms_)'))

res = []

for link in links:
    table_html = urlopen(base_url + link['href'])
    table_bs = BeautifulSoup(table_html, 'html.parser')
    table = table_bs.find('div', {'class':"mw-parser-output"}).find(
        'table', {'class':"wikitable sortable"}).tbody
    names = []
    for row in table.find_all('tr')[1:]:
        td = row.find_all('td')
        name = td[0]
        faction = td[6].text
        if faction in ['Cao Cao', 'Cao Wei', 'Wei', 'Wei dynasty']:
            faction = 'Wei'
        elif faction in ['Liu Bei', 'Shu Han', 'Shu']:
            faction = 'Shu'
        elif faction in ['Sun Quan', 'Sun Jian', 'Sun Ce', 'Eastern Wu', 'Wu']:
            faction = 'Wu'
        elif faction in ['Jin', 'Jin dynasty']:
            faction = 'Jin'
        else:
            faction = 'Other'
        name_link = name.find('a')
        if name_link and re.search('^(/wiki/)', name_link['href']): # Has link
            s = name.text
            name_en = re.compile(r'[\u4e00-\u9fa5]').split(s)[0] # English name
            name_zh = Traditional2Simplified(s[len(name_en):]) # Chinese name (Simplified)
            if name_en in name_link['title']: # Has own entry
                names.append([name_en, name_zh, faction])
    res.extend(names)

In [22]:
# Clean duplicated names
header = ['name_en', 'name_zh', 'url']
df = pd.DataFrame(columns=header, data=res)
count = defaultdict(lambda:0)
for x in df['name_en']: count[x] += 1
for key in count.keys():
    if count[key] > 1: 
        print(df[df['name_en']==key])

name_en name_zh  url
4  Empress Bian     卞皇后  Wei
5  Empress Bian     卞皇后  Wei
    name_en name_zh    url
27  Cao Jie      曹节  Other
28  Cao Jie      曹节  Other
   name_en name_zh  url
29  Cao Ju      曹矩  Wei
30  Cao Ju      曹据  Wei
    name_en name_zh  url
31  Cao Jun      曹均  Wei
32  Cao Jun      曹峻  Wei
    name_en name_zh  url
34  Cao Lin      曹霖  Wei
35  Cao Lin      曹林  Wei
    name_en name_zh  url
36  Cao Mao      曹髦  Wei
37  Cao Mao      曹茂  Wei
    name_en name_zh  url
40  Cao Rui      曹叡  Wei
41  Cao Rui      曹蕤  Wei
     name_en name_zh    url
72  Chen Shi      陈式    Shu
73  Chen Shi      陈寔  Other
      name_en name_zh url
93  Ding Feng      丁奉  Wu
94  Ding Feng      丁封  Wu
    name_en name_zh  url
95  Ding Yi      丁仪  Wei
96  Ding Yi      丁廙  Wei
      name_en name_zh    url
164  Guo Yuan      国渊    Wei
165  Guo Yuan      郭援  Other
     name_en name_zh    url
277  Liu Yan      刘焉  Other
278  Liu Yan      刘琰    Shu
    name_en name_zh  url
287   Lu Ji      陆绩   Wu
288   Lu J

In [23]:
to_delete = [4, 5, 28, 30, 32, 35, 37, 41, 73, 94, 96, 165, 277, 286, 292, 306, 392, 408, 493, 554, 584, 585, 586, 592, 602, 611, 614, 619, 620, 646, 649]
df.drop(to_delete, axis=0, inplace = True)
df.reset_index(inplace = True, drop = True)

In [24]:
# Save data
df.to_csv('/home/hengji/Documents/three_kingdoms_social_network/scrape_novel/data/people.csv', index = False)